# Statistics Challenge (Optional)

Use the `orders.csv` dataset in the same directory to complete this challenge.

**Background**:

There are exactly 100 sneaker shops on a sneaker retailing website, and each of these shops sells only one model of shoe. We want to do some analysis of the average order value (AOV). When we look at orders data over a 30 day window, we naively calculate an AOV of $3145.13. Given that we know these shops are selling sneakers, a relatively affordable item, something seems wrong with our analysis. 

**Questions**:

- What went wrong with this metric and our analysis? 

- Propose some new metrics that better represents the behavior of the stores' customers. Why are these metrics better? You can propose as many new metrics as you wish but quality heavily outweights quantity.

- Find the values of your new metrics.

- Report any other interesting findings.

Show all of your work in this notebook.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [10]:
df = pd.read_csv('orders.csv')
print(df. head())
df.dtypes

   order_id  shop_id  user_id  order_value  total_items payment_method  \
0         1       53      746          224            2           cash   
1         2       92      925           90            1           cash   
2         3       44      861          144            1           cash   
3         4       18      935          156            1    credit_card   
4         5       18      883          156            1    credit_card   

            created_at  
0  2017-03-13 12:36:56  
1  2017-03-03 17:38:52  
2   2017-03-14 4:23:56  
3  2017-03-26 12:43:37  
4   2017-03-01 4:35:11  


order_id           int64
shop_id            int64
user_id            int64
order_value        int64
total_items        int64
payment_method    object
created_at        object
dtype: object

In [33]:
# AOV naive calculaion: total_order_value/num_order_30_days
# below try calculate AOV naively
# First, convert to day-time object
df["date-time"] = pd.to_datetime(df["created_at"])
df.head()
df.dtypes
# Second, select a 30-day window
start_date = pd.to_datetime("2017-03-01 0:00:00")
end_date = pd.to_datetime("2017-03-30 23:59:59")
df_30_day = df.loc[(df['date-time'] > start_date) & (df['date-time'] < end_date)]
df_30_day.head()
AOV = df_30_day["order_value"].sum() / df_30_day["order_id"].nunique()
print(AOV)
df_30_day.iloc[df_30_day[["order_value"]].idxmax()]

3145.128


,order_id,shop_id,user_id,order_value,total_items,payment_method,created_at,date-time
15,16,42,607,704000,2000,credit_card,2017-03-07 4:00:00,2017-03-07 04:00:00


Answer for q1:
The problem with the original metric and analysis is ignoring the vast differences in total items order for each order. For example, for the first orders, the number of items ordered are 1 or 2. However, we can see in order_id 16, the number of items order is 2000, and therefore the order value is 704000, which will increase the average by a significant amount.
Even though most orders are in small amount, one large order like order_id 16 can have too much weight in naively calculating AOV, and therefore the result will not represent order value for average customers (the large orders are rare, possibly done by large firms, and shouldn't be have much weight in the result).

Answer for q2:
One better metric will be the median of order values over a 30-day window, because in that case we will look at what value most people order.
Another better metric is doing the same calculation of naive approach, but we discard extreme cases, in this scenario, large orders.

In [36]:
# First, we find metirc: median
MOV = df_30_day["order_value"].median()
MOV

284.0

In [39]:
# Second, we find metric: average but without extreme values
median_num_items_ordered = df_30_day["total_items"].median()
median_num_items_ordered

2.0

In [44]:
# Here, we consider any order above 4 items as extreme, because the median number ordered is 2
df_30_day_no_extreme = df_30_day.loc[df_30_day["total_items"] <= 4]
AOV_no_extreme = df_30_day_no_extreme["order_value"].sum() / df_30_day_no_extreme["order_id"].nunique()
AOV_no_extreme

722.3541666666666

In [46]:
# An interesting thing we notice above is that even our median metric produces a reasonable answer, the no extreme AOV
# still has a value that's higher than the normal ones we see.
# We will explore this by looking at the maximum value, because as in the naive AOV, it could change the AOV drastically.
df_30_day_no_extreme.iloc[df_30_day_no_extreme[["order_value"]].idxmax()]

,order_id,shop_id,user_id,order_value,total_items,payment_method,created_at,date-time
2548,2549,78,861,25725,1,cash,2017-03-17 19:36:00,2017-03-17 19:36:00


We noticed that even we calculate AOV without extreme cases with total items bigger than 4, we can still have extreme cases that has a extreme order value like the one shown above.
Below, I can tailor my no-extreme aov furthur by removing extreme order value too.

In [50]:
# I use 800 here because that's 4*200, where 4 is the maximum num_items we accept as normal, and 200 is the maxium
# price we accept as normal
df_30_day_no_value_or_num_extreme = df_30_day_no_extreme.loc[df_30_day_no_extreme["order_value"] <= 800]
AOV_no_value_or_num_extreme = df_30_day_no_value_or_num_extreme["order_value"].sum() / df_30_day_no_value_or_num_extreme["order_id"].nunique()
AOV_no_value_or_num_extreme

293.0644095788604

Now, we can see it produces a more reasonable AOV value